In [2]:
#mount my files to colab. and cd to mmst proj folder.
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/MyDrive/MMST-ViT

Mounted at /content/drive
/content/drive/MyDrive/MMST-ViT


In [3]:
!ps ax | grep python

     78 ?        Z      0:13 [python3] <defunct>
     79 ?        S      0:00 python3 /usr/local/bin/colab-fileshim.py
    124 ?        Sl     0:03 /usr/bin/python3 /usr/local/bin/jupyter-notebook --debug --transport="ipc" --ip=172.28.0.12 --NotebookApp.token= --port=9000 --FileContentsManager.root_dir=/ --MappingKernelManager.root_dir=/content
   2561 ?        Ssl    0:03 /usr/bin/python3 -m colab_kernel_launcher -f /root/.local/share/jupyter/runtime/kernel-eb589e9e-606a-4c37-b3cb-a7a189d1851d.json
   2619 ?        Sl     0:00 /usr/bin/python3 /usr/local/lib/python3.11/dist-packages/debugpy/adapter --for-server 43913 --host 127.0.0.1 --port 40039 --server-access-token c3f0ff736d9a9932b5e1c0cb839906f929fd784d3b70dda2bfb2ad1390506c41
   2651 ?        Sl     0:00 /usr/colab/bin/language_service --lsp_search_dirs=/datalab/web/pyright/typeshed-fallback/stdlib,/usr/local/lib/python3.10/dist-packages --language_services_request_root_url=http://172.28.0.1:8013/ --language_services_request_tim

In [12]:
#import and verify versions of installed/required packages
import torch
import timm
import einops
from PIL import Image
import sklearn
import tensorboard
import tqdm
import torchvision
import numpy as np
import pandas as pd
import h5py

print("torch:", torch.__version__)
print("timm:", timm.__version__)
print("einops:", einops.__version__)
print("Pillow:", Image.__version__)
print("sklearn:", sklearn.__version__)
print("tensorboard:", tensorboard.__version__)
print("tqdm:", tqdm.__version__)
print("torchvision:", torchvision.__version__)
print("numpy:", numpy.__version__)
print("pandas:", pandas.__version__)
print("h5py:", h5py.__version__)


!pip install datasets
from datasets import load_dataset

torch: 2.6.0+cu124
timm: 1.0.15
einops: 0.8.1
Pillow: 11.2.1
sklearn: 1.6.1
tensorboard: 2.18.0
tqdm: 4.67.1
torchvision: 0.21.0+cu124
numpy: 2.0.2
pandas: 2.2.2
h5py: 3.13.0


In [13]:
import json
import os
from tqdm import tqdm
project_path = "/content/drive/MyDrive/MMST-ViT"
sentinel_root = os.path.join(project_path, "TinyCropNet", "Sentinel-2 Imagery", "data")
hrrr_root = os.path.join(project_path, "TinyCropNet", "WRF-HRRR", "data")
usda_root = os.path.join(project_path, "TinyCropNet", "USDA Crop Dataset", "data", "Soybean")
checkpoint_path = os.path.join(project_path, "output_dir", "mmst_vit", "checkpoint-198.pth")
val_json_path = os.path.join(project_path, "data", "soybean_val_outliers.json")

#Reference: Load Model (MMST-ViT + PVTSimCLR) baseline exact to replicte their results and understand the data/research process better:
from models_mmst_vit import MMST_ViT
from models_pvt_simclr import PVTSimCLR

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

pvt = PVTSimCLR(base_model="pvt_tiny", out_dim=512, context_dim=9, pretrained=False)
model = MMST_ViT(out_dim=2, pvt_backbone=pvt, context_dim=9, dim=512, batch_size=64)

In [14]:
!nvidia-smi


Tue Apr 29 22:45:29 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   34C    P8             12W /   70W |       2MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [15]:

#made a directory for data folder to store json files
#!mkdir -p /content/drive/MyDrive/MMST-ViT/data/ #creating directory to store data file

#run config building script to convert csv data into json for soybean data
#!python /content/drive/MyDrive/MMST-ViT/config/build_config_soybean.py

#run pre-training script with loaded json files
#gpu set to cuda: 0
#!python main_pretrain_mmst_vit.py -tf /content/drive/MyDrive/MMST-ViT/data/soybean_train.json --device cuda:0

#Check if CUDA is available/workng
print("Is CUDA available: ", torch.cuda.is_available())
print("Current GPU device:", torch.cuda.current_device())
print("GPU device name:", torch.cuda.get_device_name(0))



Is CUDA available:  True
Current GPU device: 0
GPU device name: Tesla T4


In [16]:
#init. path to input file that have csv county -level info.
csv_folder = "/content/drive/MyDrive/MMST-ViT/input"

#iterate through all files [csv] in dir:
def print_csv_columns(folder):
    for filename in os.listdir(folder):
        if filename.endswith(".csv"):
            file_path = os.path.join(folder, filename)
            df = pd.read_csv(file_path)
            print(f"File: {filename}")
            print("Columns:", df.columns.tolist())
            print("-" * 40)

print("Before renaming:")
print_csv_columns(csv_folder)

# standardizing column names to maintain same FIPS Code format across all entries
for filename in os.listdir(csv_folder):
    if filename.endswith(".csv"):
        file_path = os.path.join(csv_folder, filename)
        df = pd.read_csv(file_path)
        # Check if the column "FIPS" exists and "FIPS Code" is missing, then rename.
        if "FIPS" in df.columns and "FIPS Code" not in df.columns:
            df.rename(columns={"FIPS": "FIPS Code"}, inplace=True)
            print(f"Renamed 'FIPS' to 'FIPS Code' in {filename}")
        df.to_csv(file_path, index=False)

#verifying standardized cols:
print("\nAfter renaming:")
print_csv_columns(csv_folder)


from datasets import load_dataset
dataset = load_dataset("csv", data_files="/content/drive/MyDrive/MMST-ViT/input/*.csv")
print(dataset)


#clone/loading wrf-hrr and usda yield/prod modalities from TinyCropNet data.
%cd /content/drive/MyDrive/MMST-ViT
!mkdir -p TinyCropNet #creating dir for storage
!git clone https://huggingface.co/datasets/fudong03/Tiny-CropNet TinyCropNet




Before renaming:
File: county_info_2021.csv
Columns: ['FIPS Code', 'State', 'County']
----------------------------------------
File: county_info_2022.csv
Columns: ['FIPS Code', 'State', 'County']
----------------------------------------
File: county_info.csv
Columns: ['FIPS Code', 'State', 'County']
----------------------------------------

After renaming:
File: county_info_2021.csv
Columns: ['FIPS Code', 'State', 'County']
----------------------------------------
File: county_info_2022.csv
Columns: ['FIPS Code', 'State', 'County']
----------------------------------------
File: county_info.csv
Columns: ['FIPS Code', 'State', 'County']
----------------------------------------


Generating train split: 0 examples [00:00, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['FIPS Code', 'State', 'County'],
        num_rows: 698
    })
})
/content/drive/MyDrive/MMST-ViT
fatal: destination path 'TinyCropNet' already exists and is not an empty directory.


In [17]:
#!ls -lR "/content/drive/MyDrive/MMST-ViT/TinyCropNet"
#recursively list info about each file/dir from imported tinycropnet dataset:


'''
Total : 12 files
# TinyCropNet Dataset Overview:
# ├── Sentinel-2 Imagery:
# │   └── Contains remote sensing HDF5 files organized by year (2017 to 2022) and region (e.g., IA, IL, LA, MS).
# │       - These files hold multispectral Sentinel-2 imagery data for crop monitoring.
#
# ├── USDA Crop Dataset:
# │   └── Contains CSV files for different crops (e.g., Corn, Cotton, Soybean, WinterWheat)
# │       └── Each crop folder is subdivided by year (2017 to 2022) with county-level agricultural statistics.
#
# └── WRF-HRRR Computed Dataset:
#     └── Contains CSV files generated from high-resolution weather forecasts (HRRR data)
#         └── Organized by year (2017 to 2022) and region (e.g., IA, IL, LA, MS) representing computed weather data
#            for use in crop modeling and analysis.'''

'\nTotal : 12 files\n# TinyCropNet Dataset Overview:\n# ├── Sentinel-2 Imagery:\n# │   └── Contains remote sensing HDF5 files organized by year (2017 to 2022) and region (e.g., IA, IL, LA, MS).\n# │       - These files hold multispectral Sentinel-2 imagery data for crop monitoring.\n#\n# ├── USDA Crop Dataset:\n# │   └── Contains CSV files for different crops (e.g., Corn, Cotton, Soybean, WinterWheat)\n# │       └── Each crop folder is subdivided by year (2017 to 2022) with county-level agricultural statistics.\n#\n# └── WRF-HRRR Computed Dataset:\n#     └── Contains CSV files generated from high-resolution weather forecasts (HRRR data)\n#         └── Organized by year (2017 to 2022) and region (e.g., IA, IL, LA, MS) representing computed weather data\n#            for use in crop modeling and analysis.'

In [18]:
#!df -h #checking disk space

#downloading sentinel 2 imagery per year
#100+ gb worth of hdf5 data - iowa
#2017
'''
#!wget https://huggingface.co/datasets/fudong03/Tiny-CropNet/resolve/main/Sentinel-2%20Imagery/data/2017/IA/Agriculture_19_IA_2017-01-01_2017-03-31.h5
!wget https://huggingface.co/datasets/fudong03/Tiny-CropNet/resolve/main/Sentinel-2%20Imagery/data/2017/IA/Agriculture_19_IA_2017-04-01_2017-06-30.h5
!wget https://huggingface.co/datasets/fudong03/Tiny-CropNet/resolve/main/Sentinel-2%20Imagery/data/2017/IA/Agriculture_19_IA_2017-07-01_2017-09-30.h5
!wget https://huggingface.co/datasets/fudong03/Tiny-CropNet/resolve/main/Sentinel-2%20Imagery/data/2017/IA/Agriculture_19_IA_2017-10-01_2017-12-31.h5

#2018
!wget https://huggingface.co/datasets/fudong03/Tiny-CropNet/resolve/main/Sentinel-2%20Imagery/data/2018/IA/Agriculture_19_IA_2018-01-01_2018-03-31.h5
!wget https://huggingface.co/datasets/fudong03/Tiny-CropNet/resolve/main/Sentinel-2%20Imagery/data/2018/IA/Agriculture_19_IA_2018-04-01_2018-06-30.h5
!wget https://huggingface.co/datasets/fudong03/Tiny-CropNet/resolve/main/Sentinel-2%20Imagery/data/2018/IA/Agriculture_19_IA_2018-07-01_2018-09-30.h5
!wget https://huggingface.co/datasets/fudong03/Tiny-CropNet/resolve/main/Sentinel-2%20Imagery/data/2018/IA/Agriculture_19_IA_2018-10-01_2018-12-31.h5

#2019
!wget https://huggingface.co/datasets/fudong03/Tiny-CropNet/resolve/main/Sentinel-2%20Imagery/data/2019/IA/Agriculture_19_IA_2019-01-01_2019-03-31.h5
!wget https://huggingface.co/datasets/fudong03/Tiny-CropNet/resolve/main/Sentinel-2%20Imagery/data/2019/IA/Agriculture_19_IA_2019-04-01_2019-06-30.h5
!wget https://huggingface.co/datasets/fudong03/Tiny-CropNet/resolve/main/Sentinel-2%20Imagery/data/2019/IA/Agriculture_19_IA_2019-07-01_2019-09-30.h5
!wget https://huggingface.co/datasets/fudong03/Tiny-CropNet/resolve/main/Sentinel-2%20Imagery/data/2019/IA/Agriculture_19_IA_2019-10-01_2019-12-31.h5

#2020
!wget https://huggingface.co/datasets/fudong03/Tiny-CropNet/resolve/main/Sentinel-2%20Imagery/data/2020/IA/Agriculture_19_IA_2020-01-01_2020-03-31.h5
!wget https://huggingface.co/datasets/fudong03/Tiny-CropNet/resolve/main/Sentinel-2%20Imagery/data/2020/IA/Agriculture_19_IA_2020-04-01_2020-06-30.h5
!wget https://huggingface.co/datasets/fudong03/Tiny-CropNet/resolve/main/Sentinel-2%20Imagery/data/2020/IA/Agriculture_19_IA_2020-07-01_2020-09-30.h5
!wget https://huggingface.co/datasets/fudong03/Tiny-CropNet/resolve/main/Sentinel-2%20Imagery/data/2020/IA/Agriculture_19_IA_2020-10-01_2020-12-31.h5

#2021
!wget https://huggingface.co/datasets/fudong03/Tiny-CropNet/resolve/main/Sentinel-2%20Imagery/data/2021/IA/Agriculture_19_IA_2021-01-01_2021-03-31.h5
!wget https://huggingface.co/datasets/fudong03/Tiny-CropNet/resolve/main/Sentinel-2%20Imagery/data/2021/IA/Agriculture_19_IA_2021-04-01_2021-06-30.h5
!wget https://huggingface.co/datasets/fudong03/Tiny-CropNet/resolve/main/Sentinel-2%20Imagery/data/2021/IA/Agriculture_19_IA_2021-07-01_2021-09-30.h5
!wget https://huggingface.co/datasets/fudong03/Tiny-CropNet/resolve/main/Sentinel-2%20Imagery/data/2021/IA/Agriculture_19_IA_2021-10-01_2021-12-31.h5



#2022
!wget https://huggingface.co/datasets/fudong03/Tiny-CropNet/resolve/main/Sentinel-2%20Imagery/data/2022/IA/Agriculture_19_IA_2022-01-01_2022-03-31.h5
!wget https://huggingface.co/datasets/fudong03/Tiny-CropNet/resolve/main/Sentinel-2%20Imagery/data/2022/IA/Agriculture_19_IA_2022-04-01_2022-06-30.h5
!wget https://huggingface.co/datasets/fudong03/Tiny-CropNet/resolve/main/Sentinel-2%20Imagery/data/2022/IA/Agriculture_19_IA_2022-07-01_2022-09-30.h5
!wget https://huggingface.co/datasets/fudong03/Tiny-CropNet/resolve/main/Sentinel-2%20Imagery/data/2022/IA/Agriculture_19_IA_2022-10-01_2022-12-31.h5
'''

'\n#!wget https://huggingface.co/datasets/fudong03/Tiny-CropNet/resolve/main/Sentinel-2%20Imagery/data/2017/IA/Agriculture_19_IA_2017-01-01_2017-03-31.h5\n!wget https://huggingface.co/datasets/fudong03/Tiny-CropNet/resolve/main/Sentinel-2%20Imagery/data/2017/IA/Agriculture_19_IA_2017-04-01_2017-06-30.h5\n!wget https://huggingface.co/datasets/fudong03/Tiny-CropNet/resolve/main/Sentinel-2%20Imagery/data/2017/IA/Agriculture_19_IA_2017-07-01_2017-09-30.h5\n!wget https://huggingface.co/datasets/fudong03/Tiny-CropNet/resolve/main/Sentinel-2%20Imagery/data/2017/IA/Agriculture_19_IA_2017-10-01_2017-12-31.h5\n\n#2018\n!wget https://huggingface.co/datasets/fudong03/Tiny-CropNet/resolve/main/Sentinel-2%20Imagery/data/2018/IA/Agriculture_19_IA_2018-01-01_2018-03-31.h5\n!wget https://huggingface.co/datasets/fudong03/Tiny-CropNet/resolve/main/Sentinel-2%20Imagery/data/2018/IA/Agriculture_19_IA_2018-04-01_2018-06-30.h5\n!wget https://huggingface.co/datasets/fudong03/Tiny-CropNet/resolve/main/Sentinel

In [19]:
!pwd
!ls -l
#match all iowa .h5 files by year and move to /sentinel2 directory
#17
#!mv /content/drive/MyDrive/MMST-ViT/Agriculture_19_IA_2017-*.h5 "/content/drive/MyDrive/MMST-ViT/TinyCropNet/Sentinel-2 Imagery/data/2017/IA/"
#18
#!mv /content/drive/MyDrive/MMST-ViT/Agriculture_19_IA_2018-*.h5 "/content/drive/MyDrive/MMST-ViT/TinyCropNet/Sentinel-2 Imagery/data/2018/IA/"
#19
#!mv /content/drive/MyDrive/MMST-ViT/Agriculture_19_IA_2019-*.h5 "/content/drive/MyDrive/MMST-ViT/TinyCropNet/Sentinel-2 Imagery/data/2019/IA/"

#20
#!mv /content/drive/MyDrive/MMST-ViT/Agriculture_19_IA_2020-*.h5 "/content/drive/MyDrive/MMST-ViT/TinyCropNet/Sentinel-2 Imagery/data/2020/IA/"

#21
#!mv /content/drive/MyDrive/MMST-ViT/Agriculture_19_IA_2021-*.h5 "/content/drive/MyDrive/MMST-ViT/TinyCropNet/Sentinel-2 Imagery/data/2021/IA/"
#22
#!mv /content/drive/MyDrive/MMST-ViT/Agriculture_19_IA_2022-*.h5 "/content/drive/MyDrive/MMST-ViT/TinyCropNet/Sentinel-2 Imagery/data/2022/IA/"

#verify the contents of each folder to check mv:
!ls -lR "/content/drive/MyDrive/MMST-ViT/TinyCropNet/Sentinel-2 Imagery/data/2017/IA/"
!ls -lR "/content/drive/MyDrive/MMST-ViT/TinyCropNet/Sentinel-2 Imagery/data/2018/IA/"
!ls -lR "/content/drive/MyDrive/MMST-ViT/TinyCropNet/Sentinel-2 Imagery/data/2019/IA/"
!ls -lR "/content/drive/MyDrive/MMST-ViT/TinyCropNet/Sentinel-2 Imagery/data/2020/IA/"
!ls -lR "/content/drive/MyDrive/MMST-ViT/TinyCropNet/Sentinel-2 Imagery/data/2021/IA/"
!ls -lR "/content/drive/MyDrive/MMST-ViT/TinyCropNet/Sentinel-2 Imagery/data/2022/IA/"




/content/drive/MyDrive/MMST-ViT
total 1810248
-rw------- 1 root root  48242688 Apr 17 05:15  Agriculture_19_IA_2017-04-01_2017-06-30.h5
-rw------- 1 root root 880377871 Apr 17 05:15  Agriculture_19_IA_2017-07-01_2017-09-30.h5
-rw------- 1 root root 248246272 Apr 17 05:15  Agriculture_19_IA_2017-10-01_2017-12-31.h5
-rw------- 1 root root      7393 Apr  7 21:56  attention.py
drwx------ 4 root root      4096 Jan 18  2022 'Colab Notebooks'
drwx------ 2 root root      4096 Apr  7 21:56  config
drwx------ 2 root root      4096 Apr  7 23:23  data
drwx------ 3 root root      4096 Apr  7 21:56  dataset
drwx------ 2 root root      4096 Apr 29 22:45  input
-rw------- 1 root root     19338 Apr  7 21:56  LICENSE
drwx------ 3 root root      4096 Apr  7 21:56  loss
-rw------- 1 root root     18381 Apr 15 01:05  main_finetune_mmst_vit.py
-rw------- 1 root root     12540 Apr  8 02:50  main_pretrain_mmst_vit.py
-rw------- 1 root root 676613979 Apr 19 00:10  mmst_vit_checkpoints.zip
-rw------- 1 root roo

In [21]:
#verifying retrival of h5 data by checking if filesentinel2paths can be opened.
file_path = '/content/drive/MyDrive/MMST-ViT/TinyCropNet/Sentinel-2 Imagery/data/2017/IA/Agriculture_19_IA_2017-10-01_2017-12-31.h5'

try:
    with h5py.File(file_path, 'r') as hf:
        print("File opened successfully!")
        print("Top-level groups:", list(hf.keys()))


        first_key = list(hf.keys())[0]
        print(f"Contents of {first_key}:", list(hf[first_key].keys()))

        sample_group = hf[first_key]
        first_dataset = list(sample_group.keys())[0]
        data = sample_group[first_dataset][()]
        print(f"Dataset '{first_dataset}' shape:", data.shape)

except Exception as e:
    print("Error opening file:", e)

fips = '19001'
timestamp = '2017-10-01'

with h5py.File(file_path, 'r') as hf:
    print("📂 Band names under", fips, "/", timestamp, ":")
    print(list(hf[fips][timestamp].keys()))



fips = '19001'
timestamp = '2017-10-01'
file_path = '/content/drive/MyDrive/MMST-ViT/TinyCropNet/Sentinel-2 Imagery/data/2017/IA/Agriculture_19_IA_2017-10-01_2017-12-31.h5'

with h5py.File(file_path, 'r') as hf:
    data_group = hf[f'{fips}/{timestamp}/data']
    print("Found data!")
    print("Shape of data:", data_group.shape)
    print("Dtype of data:", data_group.dtype)
    print("Example: pixel values at [0, 100, 100]:", data_group[0, 100, 100])

File opened successfully!
Top-level groups: ['19001', '19003', '19005', '19007', '19009', '19011', '19013', '19015', '19017', '19019', '19021', '19023', '19025', '19027', '19029', '19031', '19033', '19035', '19037', '19039', '19041', '19043', '19045', '19047', '19049', '19051', '19053', '19055', '19057', '19059', '19061', '19063', '19065', '19067', '19069', '19071', '19073', '19075', '19077', '19079', '19081', '19083', '19085', '19087', '19089', '19091', '19093', '19095', '19097', '19099', '19101', '19103', '19105', '19107', '19109', '19111', '19113', '19115', '19117', '19119', '19121', '19123', '19125', '19127', '19129', '19131', '19133', '19135', '19137', '19139', '19141', '19143', '19145', '19147', '19149', '19151', '19153', '19155', '19157', '19159', '19161', '19163', '19165', '19167', '19169', '19171', '19173', '19175', '19177', '19179', '19181', '19183', '19185', '19187', '19189', '19191', '19193', '19195', '19197']
Contents of 19001: ['2017-10-01', '2017-10-15', '2017-11-01', '2

In [23]:
#NDVI Formula:
#NDVI = NIR - RED / NIR + RED
#values from -1 to 1 [healthy,bare soil, water,clouds]
'''/Agriculture_19_IA_2017-10-01_2017-12-31.h5
├── '19001' (FIPS for Adair County)
│   ├── '2017-10-01'
│   ├── '2017-10-15'
│   ├── '2017-11-01'
│   ├── '2017-11-15'
│   ├── '2017-12-01'
│   └── '2017-12-15'
├── '19003'
├── ...
'''
#what band or time slices are stored inside each fips group
#shape and structure of sentinel-2 imagery tensors (often[time,band,h,w])
#which bands are included maybe (b02, b04, b08, etc.)
# Example: NDVI = (B08 - B04) / (B08 + B04)
#red = hf[fips_code]['B04'][()]
#nir = hf[fips_code]['B08'][()]
#ndvi = (nir - red) / (nir + red + 1e-6)


"/Agriculture_19_IA_2017-10-01_2017-12-31.h5\n├── '19001' (FIPS for Adair County)\n│   ├── '2017-10-01'\n│   ├── '2017-10-15'\n│   ├── '2017-11-01'\n│   ├── '2017-11-15'\n│   ├── '2017-12-01'\n│   └── '2017-12-15'\n├── '19003'\n├── ...\n"

In [24]:
#rebuilding json config to reference only iowa files due to constrained storage and disk space.
!python /content/drive/MyDrive/MMST-ViT/config/build_config_soybean.py

In [25]:
csv_path = "/content/drive/MyDrive/MMST-ViT/input/county_info_2021.csv"
df = pd.read_csv(csv_path)
print("Columns in the CSV file:", df.columns.tolist())

Columns in the CSV file: ['FIPS Code', 'State', 'County']


In [26]:
#!mv "/content/drive/MyDrive/MMST-ViT/TinyCropNet/WRF-HRRR Computed Dataset" "/content/drive/MyDrive/MMST-ViT/TinyCropNet/WRF-HRRR"
!find "/content/drive/MyDrive/MMST-ViT/TinyCropNet/WRF-HRRR/data/2021/IA" -name '*.csv'

/content/drive/MyDrive/MMST-ViT/TinyCropNet/WRF-HRRR/data/2021/IA/HRRR_19_IA_2021-01.csv
/content/drive/MyDrive/MMST-ViT/TinyCropNet/WRF-HRRR/data/2021/IA/HRRR_19_IA_2021-02.csv
/content/drive/MyDrive/MMST-ViT/TinyCropNet/WRF-HRRR/data/2021/IA/HRRR_19_IA_2021-03.csv
/content/drive/MyDrive/MMST-ViT/TinyCropNet/WRF-HRRR/data/2021/IA/HRRR_19_IA_2021-04.csv
/content/drive/MyDrive/MMST-ViT/TinyCropNet/WRF-HRRR/data/2021/IA/HRRR_19_IA_2021-05.csv
/content/drive/MyDrive/MMST-ViT/TinyCropNet/WRF-HRRR/data/2021/IA/HRRR_19_IA_2021-06.csv
/content/drive/MyDrive/MMST-ViT/TinyCropNet/WRF-HRRR/data/2021/IA/HRRR_19_IA_2021-07.csv
/content/drive/MyDrive/MMST-ViT/TinyCropNet/WRF-HRRR/data/2021/IA/HRRR_19_IA_2021-08.csv
/content/drive/MyDrive/MMST-ViT/TinyCropNet/WRF-HRRR/data/2021/IA/HRRR_19_IA_2021-09.csv
/content/drive/MyDrive/MMST-ViT/TinyCropNet/WRF-HRRR/data/2021/IA/HRRR_19_IA_2021-10.csv
/content/drive/MyDrive/MMST-ViT/TinyCropNet/WRF-HRRR/data/2021/IA/HRRR_19_IA_2021-11.csv
/content/drive/MyDriv

In [27]:
# Running the pre-training script with the following h-parameters
# so that our MMST Model / VIT backbone
# learns all the supervised data and patterns [200 epcohs to see it many times]
# with a constrative loss func
# on both csv and images info for the state of iowa

# batch_size=32  : Process 32 samples per GPU in each training iteration.
# embed_dim=512    : The dimension of the embedding/output vector for the backbone.
# epochs=200       : Total number of training epochs.
# model='pvt_tiny'  : The name of the backbone model architecture (PVT tiny variant).
# accum_iter=1     : Number of iterations over which gradients are accumulated.
# input_size=224      : Input image resolution (height and width).
# norm_pix_loss=False    : Whether to use per-patch normalized pixel loss (disabled here).
# weight_decay=0.05     : Weight decay parameter for the optimizer.
# layer_decay=0.75       : Factor used for decaying learning rates across transformer layers.
# lr=None and blr=0.001   : Learning rate is derived from the base learning rate (blr) scaled by the effective batch size.
# min_lr=0.0          : The minimum learning rate bound.
# warmup_epochs=20    : Number of epochs used to warm up the learning rate.

# seed=0          : Random seed for reproducibility.
# resume=''              : Path to a checkpoint file from which to resume training (empty means training from scratch).
# start_epoch=0          : Training will start from epoch 0.
# num_workers=1         : Number of worker processes for the data loader.
# pin_mem=True         : Whether to use pinned memory in the data loader for more efficient GPU transfers.
# world_size=1            : Number of processes in distributed training (1 here means no distributed training).
# local_rank=-1           : Local process rank (not used since distributed is False).
# dist_on_itp=False       : Flag for distributed training over intra-node processes (disabled here).
# save_freq=5            : Save checkpoints every 5 epochs.
# distributed=False      : Disable distributed training (runing on a single device).

# output_dir='./output_dir/pvt_simclr' : Directory where the model checkpoints and logs will be saved.
# log_dir='./output_dir/pvt_simclr'       : Directory where tensorboard logs will be written.



In [ ]:
!python main_pretrain_mmst_vit.py \
    --data_file "/content/drive/MyDrive/MMST-ViT/data/soybean_train.json" \
    --root_dir "/content/drive/MyDrive/MMST-ViT/TinyCropNet" \
    --device cuda:0

2025-04-08 23:26:48.124642: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1744154808.152047   56092 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1744154808.159144   56092 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-04-08 23:26:48.182950: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
/usr/local/lib/python3.11/dist-packages/timm/optim/optim_factory.py:7: FutureWarning: Importing from timm.optim.optim

In [ ]:
#Succesfully trained the pre-tuned model on 200 epochs.
# Avg Loss: 2.2173
# ETA: 0 [at end]
# Converged PVTSIMclr model

In [ ]:
#!pwd
#!ls -l

/content/drive/MyDrive/MMST-ViT
total 1149435
-rw------- 1 root root  48242688 Apr 17 05:15  Agriculture_19_IA_2017-04-01_2017-06-30.h5
-rw------- 1 root root 880377871 Apr 17 05:15  Agriculture_19_IA_2017-07-01_2017-09-30.h5
-rw------- 1 root root 248246272 Apr 17 05:15  Agriculture_19_IA_2017-10-01_2017-12-31.h5
-rw------- 1 root root      7393 Apr  7 21:56  attention.py
drwx------ 2 root root      4096 Jan 18  2022 'Colab Notebooks'
drwx------ 2 root root      4096 Apr  7 21:56  config
drwx------ 2 root root      4096 Apr 17 05:17  data
drwx------ 2 root root      4096 Apr  7 21:56  dataset
drwx------ 2 root root      4096 Apr 17 05:16  input
-rw------- 1 root root     19338 Apr  7 21:56  LICENSE
drwx------ 2 root root      4096 Apr  7 21:56  loss
-rw------- 1 root root     18381 Apr 15 01:05  main_finetune_mmst_vit.py
-rw------- 1 root root     12540 Apr  8 02:50  main_pretrain_mmst_vit.py
drwx------ 2 root root      4096 Apr  9 03:43  MMST_VIT_implementation
-rw------- 1 root root

In [ ]:
#!python main_finetune_mmst_vit.py
#Fine tune script:
'''
Deals with having to fine tune an MMST viT model using 3 modals
major components;
- uses argeparse to define parametesr: batch, eta, model backbone, dataset file paths and training settings.
- loads all 3 datasets for trainig and validation,
  - for each set it creates data loaders and splits data across processes with distributed sampler
-uses pvtsimclr (loaded from checkpoint)
- main mmmst model is created
- training loop is implemented in train_one_epoch:
  - learning rate is adjusted per iteration
  - batches are processed by iterating over 3 datasets
  - loss is computed using mse and scaled for gradient accum.
  - metrics are logged using metric logger

evaluation:
evaluation routine that ocntinuously computes performance metrics like R2 and RMSE, correlation
- on the validation sets and prints the results.'''

'\nDeals with having to fine tune an MMST viT model using 3 modals\nmajor components;\n- uses argeparse to define parametesr: batch, eta, model backbone, dataset file paths and training settings.\n- loads all 3 datasets for trainig and validation,\n  - for each set it creates data loaders and splits data across processes with distributed sampler\n-uses pvtsimclr (loaded from checkpoint)\n- main mmmst model is created\n- training loop is implemented in train_one_epoch:\n  - learning rate is adjusted per iteration\n  - batches are processed by iterating over 3 datasets\n  - loss is computed using mse and scaled for gradient accum.\n  - metrics are logged using metric logger\n\nevaluation:\nevaluation routine that ocntinuously computes performance metrics like R2 and RMSE, correlation\n- on the validation sets and prints the results.'

In [ ]:
#fixing datatime package to use datatime modules.
#!pip install datatime
#now running evaluation with checkpoint 200 epoch
!python main_finetune_mmst_vit.py \
    --data_file_val "/content/drive/MyDrive/MMST-ViT/data/soybean_val.json" \
    --root_dir "/content/drive/MyDrive/MMST-ViT/TinyCropNet" \
    --device cuda:0 \
    --eval \
    --resume "/content/drive/MyDrive/MMST-ViT/output_dir/mmst_vit/checkpoint-198.pth"

2025-04-15 01:13:34.702492: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-04-15 01:13:34.719718: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1744679614.741439   73639 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1744679614.748095   73639 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-04-15 01:13:34.769992: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instr

In [ ]:
'''results interpretation

- sucessully resumed from the latest checkpoint. pth
- using mmst-vit model with pvt-tiny as the backbone and our modality data
- evaluation year is 2022 on 2 counties 19117 and 19135.


Final metrics:
RMSE: 2.78
R²: 1.00
Corr: 1.00

R² of 1.00 and Correlation of 1.00
mean the model predictions are nearly a
perfect fit to your ground truth labels
for the validation set.

RMSE of 2.78 is low
(assuming your crop yield units make that a meaningful value
— for soybeans, yield is usually in bushels/acre,
so that’s strong).


Model has successfully fine-tuned
 and is showing excellent predictive performance
 on the validation counties (FIPS 19117 and 19135).

This suggests that:
The model learned meaningful patterns during pretraining
Fine-tuning adapted it well to your labeled yield data

'''

In [ ]:
#read the log files from your output directory and
#provide you with interactive visualizations of your training metrics.
#!tensorboard --logdir=./output_dir/mmst_vit
#creating a subdirectory to store one event file and point tensorboard there first.
#!mkdir single_event_dir
#!cp /content/drive/MyDrive/MMST-ViT/output_dir/mmst_vit/events.out.tfevents.1744679587.94cafaefc4da.73448.0 /content/single_event_dir/
#!tensorboard --logdir=/content/single_event_dir --host=0.0.0.0 --port=6006


In [ ]:
'''
evaluation frm last checkpoint produced:
- rmse 2.78
- r2 ; 1
corr ; 1
'''


In [ ]:
'''

parameter groups 0 and 1 in our optimizer outputs mean that our model
has been split into two diff groups based on how we are applying weight decay
0 and 0.05 to help avoid overfitting and penalizing regularizing parameters that are prone
to overfittting like bias/norm weights vs convolutional and fully connected layers that need it

Epoch and Iteration Information
For each epoch, our log shows two iterations (2 counties fips 19117, 19135)
- lrate starts low, at epoch 100 it is 0.000172, and falls to 0
-training loss starts at 102, decreases to 0.0058 by epoch 195, gettin closer to the target vals.
-rmse vals indicate the average prediction error in the same units as my target variable.
- logs consistently show an rmse of 2.84 from around epoch 60 and up.
r2 = 1.00, equals a perfect fit (predictions explain all the variance in the true data)
- almost too perfect considering ral-data settings, it means the model predictions on
- the validation are set very close to the target values, and are very close to the targets or metric is not computed.
-means model predictions on the validation set are very close to the targets or metric is not computed rihgt.

correlation = 1, equals perfect relationship between predictions and true vals.



In [ ]:

!pwd
!cd project_repo


/content/drive/MyDrive/MMST-ViT


In [ ]:
#Roles of pretraining vs fine tuning:
'''
pre train is used to learn general feature representations (self and unsupervised manner) on a large dataset.
this idea is used to initalize the model or backbone with weights that have already learned useful or multimodal features.
- we pretrained my model (like the pvtsimclr backbone)


after pretraining, we run fine tune on soybean_train data

- this is used to predict soybean related outcomes.
this step adjusts the pre-trained weights by training on labeled data so that the model
learns more outside of what was captured by pretraining
- basically tailors our model to exact labels and dist of our soybean data.

evaluation output: every epoch printed shows us the quality of our soybean val
- with metrics like rmse and r2, loss, learning rate

- use soybean val for evaluation only.

- consider deployinh model for further testing on a final hold out or test set.

-- model selection or hypterparam tuning:
if validation performance is good, consider adjusting learning rate, epochs, batch size etc.
- modifying architecture or fine tuning strategy (freezing/unfreezing certain layers)
- increasing data quality or quantity through data agumentation


- After fine tuning
- we have to run a evaluation or inference script on a seperate test dataset, not the training or valid sets,
to get the unbiased perforamnce needed for estimation of valdidation.
'''


In [ ]:
'''
epoch
00:03:30.066567] Epoch: [108]  [ 0 / 1]  FIPS Code: 19117  Number of Grids: 8  Max Mem: 4275
[00:03:30.209966] Epoch: [108]  [ 1 / 1]  FIPS Code: 19135  Number of Grids: 8  Max Mem: 4275
[
  means we in epoch 108, and 0/1 means which batch we are in (one iteation per dataset)

batch size in the context of finetuning, logs,etcc.

batch size of 1 returns one sample per iteration
Batch size controls the number of samples processed before updating model weights and affects the stability of gradient estimates.

our gradient accumulation and batch size uses 64 - effective for scaling learning rate and stabilie training
[23:44:51.318968] base lr: 1.00e-03
[23:44:51.318986] actual lr: 2.50e-04
[23:44:51.318996] accumulate grad iterations: 1
[23:44:51.319008] effective batch size: 64
'''

In [ ]:
'''
Config-> build_config_soybean

generates structured json configuration files that describe our data
selects data only from iowa based on fips codes
includes metadata like fips, year, county, state names..

- constructs file paths for our hrrr,usda, and imagery data files.

-config script then builds objects for all relevant counties
in output files - soybean_train and val.json that are used by our finetuning script for training and eval.




'''

In [ ]:
'''
Pre-training - PVTSimClr

- employs self-supervised learning methods (simclr)
to learn robust and feature representation on large amounts of data to move to fine tuning.

pvtsimcl log.txt

json record that records key metrics at each epoch before finetuning.
'''


In [ ]:
'''
MMST - main tune logs of training process:
each line of log.txt is a json record that records
key training metrics per epoch.

- train_lr -> learning rate per epoch (increases than stabilizes)
- train-loss -> average training loss over all iterations in that epoch
- shows you how well the model is learning, lower loss = good and plateau
- epoch: each epoch is one complete pass over training data
'''


In [ ]:
#Zipping 2 modalities of usda, wrf-hrr data for jupyter notebook, allows me to directly reference data locally in my downloads.
!zip -r mmst_vit_1.zip /content/drive/MyDrive/MMST-ViT/data \
                           /content/drive/MyDrive/MMST-ViT/TinyCropNet/

'''
!zip -r mmst_vit_checkpoints.zip /content/drive/MyDrive/MMST-ViT/output_dir/mmst_vit/checkpoint-198.pth \
                                 /content/drive/MyDrive/MMST-ViT/output_dir/pvt_simclr/checkpoint-199.pth
'''


Scanning files ...
  adding: content/drive/MyDrive/MMST-ViT/data/ (stored 0%)
  adding: content/drive/MyDrive/MMST-ViT/data/soybean_val.json (deflated 92%)
  adding: content/drive/MyDrive/MMST-ViT/data/soybean_train.json (deflated 92%)
  adding: content/drive/MyDrive/MMST-ViT/TinyCropNet/ (stored 0%)
  adding: content/drive/MyDrive/MMST-ViT/TinyCropNet/.git/ (stored 0%)
  adding: content/drive/MyDrive/MMST-ViT/TinyCropNet/.git/branches/ (stored 0%)
  adding: content/drive/MyDrive/MMST-ViT/TinyCropNet/.git/hooks/ (stored 0%)
  adding: content/drive/MyDrive/MMST-ViT/TinyCropNet/.git/hooks/update.sample (deflated 68%)
  adding: content/drive/MyDrive/MMST-ViT/TinyCropNet/.git/hooks/pre-receive.sample (deflated 40%)
  adding: content/drive/MyDrive/MMST-ViT/TinyCropNet/.git/hooks/pre-applypatch.sample (deflated 38%)
  adding: content/drive/MyDrive/MMST-ViT/TinyCropNet/.git/hooks/push-to-checkout.sample (deflated 55%)
  adding: content/drive/MyDrive/MMST-ViT/TinyCropNet/.git/hooks/commit-msg.

'\n!zip -r mmst_vit_checkpoints.zip /content/drive/MyDrive/MMST-ViT/output_dir/mmst_vit/checkpoint-198.pth                                  /content/drive/MyDrive/MMST-ViT/output_dir/pvt_simclr/checkpoint-199.pth \n'